In [ ]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
%pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import os
MODEL = "gpt-4o"

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "API_KEY"))

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.




    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return output strictly from the text.
    If there are no points/arguments in the text, answer NONE.


     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be multiple answers, single answers or none in the text, same for the arguments.
    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'Barriers to success in teaching/student life?'
theme = "research"
text = '''
One user  said:
That one barrier is lack of opportunities for students to have a social interactions and activities in the region.

Another user said:

Another author said that the volunteering program SEP provides social interaction opportunities for students availability of volunteering program.
There is a program encouraging volunteering among UM6P students, to work with associations, and do some social work, the initiative is called SEP, we count today more than 40 students who participated. Another point is that the lack of opportunity is a chance for creation; communication is key. The absence of an opportunity is an opportunity in itself to be created UM6P offers many opportunities to students. There are departments who their activity is to help students launch clubs, social initiatives, and start-ups.... So maybe this is a lack of communication challenge rather than an absence of opportunities.

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: That one barrier is lack of opportunities for students to have a social interactions and activities in the region.
Classification of Answer: BARRIER

Answer: Another author said that the volunteering program SEP provides social interaction opportunities for students availability of volunteering program.
Classification of Answer: SOLUTION

Argument: There is a program encouraging volunteering among UM6P students, to work with associations, and do some social work, the initiative is called SEP, we count today more than 40 students who participated.
Classification of Argument: PRO

Argument: Another point is that the lack of opportunity is a chance for creation; communication is key. The absence of an opportunity is an opportunity in itself to be created UM6P offers many opportunities to students.
Classification of Argument: PRO

Argument: There are departments who their activity is to help students launch clubs, social initiatives, and start-ups.... So maybe this is a lack of com

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.


     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'what are the problem areas in the SCI masters program?'
theme = "research"
text = '''
User 1 said :
 too much homework

User 2 said this in response to user 1 :
 Homework is good. Some weeks are homework-free while some weeks are homework-overwhelming. If only there is a way professors can coordinate to distribute workloads across the semeste
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)


Answer: too much homework
Classification of Answer: BARRIER

Argument: Homework is good. Some weeks are homework-free while some weeks are homework-overwhelming. If only there is a way professors can coordinate to distribute workloads across the semester.
Classification of Argument: PRO


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.


    The output should be in the following format:
    A1:
    A1 is a (state whether it is a solution or metric or a barrier)
    PRO:
    CON:
     ...
    NB: There might be multiple answers, single answers or none in the text, same for the arguments.
    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
User 1 said :
 Content of cognitive science classes should be re-evaluated
User 2 said this in response to user 1 :
We should organize the course content so that students interested in pursuing PhD in cognitive science can do so without fear. How? Hopefully, the course should give a background in cognitive science and touch on all areas of this field, such as psychology, neuroscience, linguistics, etc. If not all, at least psychology, neuroscience, and philosophy might be very important.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

A1: Content of cognitive science classes should be re-evaluated
A1 is a BARRIER: Content of cognitive science classes should be re-evaluated
PRO: NONE
CON: NONE

A2: We should organize the course content so that students interested in pursuing PhD in cognitive science can do so without fear.
A2 is a SOLUTION: We should organize the course content so that students interested in pursuing PhD in cognitive science can do so without fear.
PRO: Hopefully, the course should give a background in cognitive science and touch on all areas of this field, such as psychology, neuroscience, linguistics, etc. If not all, at least psychology, neuroscience, and philosophy might be very important.
CON: NONE


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.

    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
User 1 said :
					There is need for specialization

User 2 said :
						I don't think it is a good thing, allow student to explore and be a tool for many jobs.


'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: There is need for specialization
Classification of Answer: SOLUTION

Argument: I don't think it is a good thing, allow student to explore and be a tool for many jobs.
Classification of Argument: CON


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
User 1 said :
					We do not benefit from some classes
User 2 said :

						I disagree with scientific English being boring; it would even be more beneficial if it were a compulsory course throughout the master's program.


'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: We do not benefit from some classes
Classification of Answer: BARRIER

Argument: I disagree with scientific English being boring; it would even be more beneficial if it were a compulsory course throughout the master's program.
Classification of Argument: CON


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
					No midterm break


						this is the most important, because as of now. we are already worn out due to many readings, projects and assignment.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: No midterm break
Classification of Answer: BARRIER

Argument: this is the most important, because as of now. we are already worn out due to many readings, projects and assignment.
Classification of Argument: PRO


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
user 1 said :
					too much homework

User 2 said this in response to user 1 :
						If length of lecture is reduced, there will be time to do home work. Everything is just too much.'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: too much homework
Classification of Answer: BARRIER

Argument: If length of lecture is reduced, there will be time to do home work.
Classification of Argument: PRO

Argument: Everything is just too much.
Classification of Argument: PRO


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What are our pain points?'
theme = "research"
text = '''
User 1 said :
					too many hours of classes per week
User 2 said this in response to user 1 :

						We could have at most two lectures per day

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: too many hours of classes per week
Classification of Answer: BARRIER

Argument: We could have at most two lectures per day
Classification of Argument: PRO


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = 'What solutions should we implement'
theme = "research"
text = '''
user 1 said this :
					Research as means to foster innovation in SCI Master's Program

User 2 said this in response to user 1:
						Master's students should be involved in the research works in SCI. It may also be necessary that they start developing practical knowledge early. I guess the research project should be the student's contribution to the community. It will be most useful after the six-month internship. They need to have the choice to work on relevant subjects that they are interested in.

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Research as means to foster innovation in SCI Master's Program
Classification of Answer: SOLUTION

Answer: Master's students should be involved in the research works in SCI.
Classification of Answer: SOLUTION

Argument: It may also be necessary that they start developing practical knowledge early.
Classification of Argument: PRO

Argument: I guess the research project should be the student's contribution to the community.
Classification of Argument: PRO

Argument: It will be most useful after the six-month internship.
Classification of Argument: PRO

Argument: They need to have the choice to work on relevant subjects that they are interested in.
Classification of Argument: PRO


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P's success in the area of research?'''
theme = "research"
text = '''
User 1 said :
 The amount of research funding. Lots of funding implies that we are doing good research; More funding help to do more research; High funding allow conducting the research more efficiently and allows for the use of modern technology.

User 2 said:
Its not always true,  managers don’t know how to manage well as it is better to choose the right managers.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The amount of research funding.
Classification of Answer: METRIC

Argument: Lots of funding implies that we are doing good research; More funding help to do more research; High funding allow conducting the research more efficiently and allows for the use of modern technology.
Classification of Argument: PRO

Argument: Its not always true, managers don’t know how to manage well as it is better to choose the right managers.
Classification of Argument: CON


In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.


    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.



     The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P's success in the area of research?'''
theme = "research"
text = '''
User  1  said:
The prestige of the funding agency. World-class funding agencies offers network and international recognition.
Increased Funding Opportunities. This funding can enable the university to pursue ambitious projects and initiatives that might otherwise be unattainable. International collaborations indicate good research capability and impact.  Attracting international funds is a positive reputation.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The prestige of the funding agency.
Classification of Answer: METRIC

Argument: World-class funding agencies offer network and international recognition.
Classification of Argument: PRO

Answer: Increased Funding Opportunities.
Classification of Answer: METRIC

Argument: This funding can enable the university to pursue ambitious projects and initiatives that might otherwise be unattainable.
Classification of Argument: PRO

Argument: International collaborations indicate good research capability and impact.
Classification of Argument: PRO

Argument: Attracting international funds is a positive reputation.
Classification of Argument: PRO


#Test case 16

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P's success in the area of research?'''
theme = "research"
text = '''
User 1 said :

 Management, ie, the researcher must supervise the students. Without good supervision, the student cannot follow the right paths, supervision is critical in training future researchers. The supervisor must be in the field .The number of supervised students and the quality of the research output should be assessed .

User 2 said in response to user 1 :

 The supervision depends on the student, his will and his motivation, a supervisor,first of all must be a psychologically stable person with an open mind. the strength of our supervision showcases the ability to transfer knowledge to future researchers and also ensure purpetuity   -A student will do his best if he wants with light supervision, A student will do his minimum if he wants even with forced supervision.

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The number of supervised students and the quality of the research output should be assessed.
Classification of Answer: METRIC

Argument: Without good supervision, the student cannot follow the right paths, supervision is critical in training future researchers.
Classification of Argument: PRO

Argument: The supervisor must be in the field.
Classification of Argument: PRO

Argument: The supervision depends on the student, his will and his motivation.
Classification of Argument: CON

Argument: A student will do his best if he wants with light supervision, A student will do his minimum if he wants even with forced supervision.
Classification of Argument: CON


#Test case 17

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P's success in the area of research?'''
theme = "research"
text = '''
User 1 said :
The total number of research papers produced. I see that the more the university research activity is high, the more papers are produced. So it is a good success indicator .

User 2 said this in response to user 2 :
The total number of research papers does not mean much, it is mainly the quality papers that matters, ie , impact factor   . The focus on number may lead to shallow content . The number is not important if not relevant papers In some fields, the number is not a KPI (ei Humanities)

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The total number of research papers produced.
Classification of Answer: METRIC

Argument: I see that the more the university research activity is high, the more papers are produced. So it is a good success indicator.
Classification of Argument: PRO

Answer: The total number of research papers does not mean much, it is mainly the quality papers that matters, ie, impact factor.
Classification of Answer: METRIC

Argument: The focus on number may lead to shallow content.
Classification of Argument: CON

Argument: The number is not important if not relevant papers.
Classification of Argument: CON

Argument: In some fields, the number is not a KPI (ei Humanities).
Classification of Argument: CON


#TEST CASE 18

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and separates different points in one sentence"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P's success in the area of research?'''
theme = "research"
text = '''
User 1 said :

It is important to be evaluated by international experts and to evaluate research structures and groups every 5 years.

User 2 said this in response to user 1:
Yes, as long as the senior members of the specific scientific community are consulted.
=>

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: It is important to be evaluated by international experts and to evaluate research structures and groups every 5 years.
Classification of Answer: METRIC

Argument: Yes, as long as the senior members of the specific scientific community are consulted.
Classification of Argument: PRO


#Test Case 19

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
Scientific/Financial/Hiring. Examples: to hire interns, a professor should not need to go through a lengthy process with HR. Paying small fees for conference registrations requires a lengthy process with Finance.
This will help to match the project with the right person in time without delay due to HR procedures.

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)




Answer: to hire interns, a professor should not need to go through a lengthy process with HR.
Classification of Answer: BARRIER
Argument: This will help to match the project with the right person in time without delay due to HR procedures.
Classification of Argument: PRO

Answer: Paying small fees for conference registrations requires a lengthy process with Finance.
Classification of Answer: BARRIER
Argument: NONE


#Test Case 20

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Lack of Internal collaboration.

User 2 said this in reponse to user 1:

Yes, lots of Individualism, Encouraging internal collaboration increases collective intelligence  because of the diversity of Um6p’s departements, they should collaborate to find research problems solutions there is very limited collaboration between departments   just sharing the infomation or activity domains of the department will creat more possibilities of collaboration.

User 3 said to contribute to the conversation:
Avoid conflicts of interest by avoiding that the lead program is in direct "competition" with the members of its entity. Interdisciplinary projects help moderate the effect of this type of conflict. This dynamism was created by calls for multidisciplinary projects, which is a very good initiative but remains insufficient.

'''

extract_points_find_clusters_and_analyze_arguments(theme, text)


Answer: Lack of Internal collaboration.
Classification of Answer: BARRIER

Answer: Encouraging internal collaboration increases collective intelligence because of the diversity of Um6p’s departments, they should collaborate to find research problems solutions there is very limited collaboration between departments just sharing the information or activity domains of the department will create more possibilities of collaboration.
Classification of Answer: SOLUTION

Argument: Encouraging internal collaboration increases collective intelligence because of the diversity of Um6p’s departments.
Classification of Argument: PRO

Argument: They should collaborate to find research problems solutions.
Classification of Argument: PRO

Argument: There is very limited collaboration between departments.
Classification of Argument: CON

Argument: Just sharing the information or activity domains of the department will create more possibilities of collaboration.
Classification of Argument: PRO

Answer: A

#Test case 21

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Creation of a large number of departments working on the same research subjects. (UM6P internal conflict of interest), More value must be given to PhD students to boost their mood to work and publish (high price of accommodation and registration fees)

User 2 said this in response to user 1 :

Creating a large number of departments working on the same research topics within UM6P can lead to internal conflicts. PhD have excellent conditions. students have very good working conditions that other students do not have, neither in Morocco nor in developed countries. students have very good working conditions that other students do not have, neither in Morocco nor in developed countries.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)


Answer: Creation of a large number of departments working on the same research subjects.
Classification of Answer: BARRIER
Argument: (UM6P internal conflict of interest)
Classification of Argument: PRO

Answer: More value must be given to PhD students to boost their mood to work and publish (high price of accommodation and registration fees)
Classification of Answer: BARRIER
Argument: PhD have excellent conditions. students have very good working conditions that other students do not have, neither in Morocco nor in developed countries.
Classification of Argument: CON


#Test case 22

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said  :

Lack of competent scientific advice.

For research to work there must be a competent scientific council of the university elected by the faculty members and therefore by the researchers themselves
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Lack of competent scientific advice.
Classification of Answer: BARRIER

Answer: For research to work there must be a competent scientific council of the university elected by the faculty members and therefore by the researchers themselves
Classification of Answer: SOLUTION


#Test case 23

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
Making all parties feel included, Researchers, PhD student, staff and engineer in research. By first covering critical necessities housing accessibility life style.. and by strategically planning activities events and  that will maximise the productivity and motivation in one hand and the opportunity to link and learn together.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Making all parties feel included, Researchers, PhD student, staff and engineer in research.
Classification of Answer: SOLUTION
Argument: By first covering critical necessities housing accessibility life style.. and by strategically planning activities events and that will maximise the productivity and motivation in one hand and the opportunity to link and learn together.
Classification of Argument: PRO


#Test case 24

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said:

Lack of evaluation skills. Need for more communication and involvement of the staff in important decisions. Lack of real sense of leadership.
Program managers must be evaluated and trained in the management of structures. A good researcher is not always a good manager.
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Lack of evaluation skills.
Classification of Answer: BARRIER

Answer: Need for more communication and involvement of the staff in important decisions.
Classification of Answer: BARRIER

Answer: Lack of real sense of leadership.
Classification of Answer: BARRIER

Answer: Program managers must be evaluated and trained in the management of structures.
Classification of Answer: SOLUTION

Argument: A good researcher is not always a good manager.
Classification of Argument: PRO


#Test case 25

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said:
Use competition as a driving force to advance research.

User 2 said this in reponse to user 1:

The University is still young to compete, it is better to evaluate the yearly performance. Evaluating the yearly performance of the University until it is on the same stage of universities of reference on three levels (Morocco, Africa, Worldwide)
'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Use competition as a driving force to advance research.
Classification of Answer: SOLUTION

Answer: The University is still young to compete, it is better to evaluate the yearly performance.
Classification of Answer: BARRIER

Argument: Evaluating the yearly performance of the University until it is on the same stage of universities of reference on three levels (Morocco, Africa, Worldwide)
Classification of Argument: PRO


#Test case 26

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Purchasing delay affect research project .
This is very important. Put in place procedures/processes to limit delays and ensure equipment received is the same as the one ordered. Fluidity of the process. Find a way to make purchasing products easier. Spent a lot of time waiting of Chemical Items Glassware &amp Consumables.

User 2 said this in reponse to user 1:

This process should be as smooth as possible. In European universities, the project manager is responsible for recruiting the postdoc he wants and the only time he goes to HR is for the Ex contract (chemical products).'''

extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Purchasing delay affect research project.
Classification of Answer: BARRIER

Argument: This is very important. Put in place procedures/processes to limit delays and ensure equipment received is the same as the one ordered. Fluidity of the process. Find a way to make purchasing products easier. Spent a lot of time waiting of Chemical Items Glassware &amp Consumables.
Classification of Argument: PRO

Answer: This process should be as smooth as possible.
Classification of Answer: SOLUTION

Argument: In European universities, the project manager is responsible for recruiting the postdoc he wants and the only time he goes to HR is for the Ex contract (chemical products).
Classification of Argument: PRO


#Test case 27

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Lack of funding for junior faculty
Insufficient (access to) startup funds means new projects can’t get started,  Junior faculty should have enough funding to jump start their research Continuity

There Should be support for junior professors because their impact has longer time as they will eventually spend longer time doing research based at UM6P which should strengthen the ecosystem .

User 2 said this in response to user 1:
We can easly find an opportunity if we have solid research proposal ,the young researcher must learn to work in a team with his or her hierarchical manager
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Lack of funding for junior faculty
Classification of Answer: BARRIER
Argument: Insufficient (access to) startup funds means new projects can’t get started
Classification of Argument: PRO

Answer: Junior faculty should have enough funding to jump start their research Continuity
Classification of Answer: SOLUTION
Argument: There Should be support for junior professors because their impact has longer time as they will eventually spend longer time doing research based at UM6P which should strengthen the ecosystem.
Classification of Argument: PRO

Answer: We can easily find an opportunity if we have solid research proposal
Classification of Answer: SOLUTION
Argument: the young researcher must learn to work in a team with his or her hierarchical manager
Classification of Argument: PRO


#test case 28

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''

User 1 said :

New departments that need time to mature and have all necessary equipment.

User 2 said this in response to user 1 :

Yes, departments should be evaluated based on their maturity level.
it will absolutely take time and hard work to build a pattern for processing , It takes time for new departments to develop and mature. During this period, there may be a lack of experienced people, which can impact the quality and quantity of research output.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: New departments that need time to mature and have all necessary equipment.
Classification of Answer: BARRIER

Argument: it will absolutely take time and hard work to build a pattern for processing, It takes time for new departments to develop and mature. During this period, there may be a lack of experienced people, which can impact the quality and quantity of research output.
Classification of Argument: PRO


#Test case 29

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in research?'''
theme = "research"
text = '''
User 1 said:

	Via partnership with different sectors such as industry, administration, NGO, etc. And give access to the University for all the stakeholders (companies, other universities, etc...)
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Via partnership with different sectors such as industry, administration, NGO, etc.
Classification of Answer: SOLUTION

Answer: give access to the University for all the stakeholders (companies, other universities, etc...)
Classification of Answer: SOLUTION

Argument: NONE
Classification of Argument: NONE


#Test case 30

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Access to high quality students/ Postdoc in Morocco and Africa

The choice of highly motivated ambitious students since the recruitment stage and motivate them more Absolutely. Should put in place outreach activities to attract talented students quality students

The Lydex experience shows this fact that it is very difficult to find good quality students and PostDocs,

In africa and in morocco, high quality students and postdocs are everywhere We shouldn’t blame students for our unsuccessful research.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The choice of highly motivated ambitious students since the recruitment stage and motivate them more
Classification of Answer: SOLUTION
Argument: Absolutely. Should put in place outreach activities to attract talented students quality students
Classification of Argument: PRO

Answer: It is very difficult to find good quality students and PostDocs
Classification of Answer: BARRIER
Argument: NONE

Answer: We shouldn’t blame students for our unsuccessful research.
Classification of Answer: BARRIER
Argument: NONE


#Test case 31

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :

Lack of clear procedures for career development.

For staff who do PhD at um6p : many disadvantages  Belonging to UM6P

User 2 in reponse to user 1 said :
The staff who do PhD do not have a vision of their future after 4 years and also they do not receive an annual bonus even if they keep their tasks in full or in part and also they do not benefit salary increase throughout the PhD period. Clear career development pathways help with strengthening the academic belonging to his/her institution.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Lack of clear procedures for career development.
Classification of Answer: BARRIER

Answer: For staff who do PhD at UM6P: many disadvantages belonging to UM6P.
Classification of Answer: BARRIER

Argument: The staff who do PhD do not have a vision of their future after 4 years and also they do not receive an annual bonus even if they keep their tasks in full or in part and also they do not benefit salary increase throughout the PhD period.
Classification of Argument: PRO

Argument: Clear career development pathways help with strengthening the academic belonging to his/her institution.
Classification of Argument: PRO


#Test case 32

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''
User 1 said :
Trying to merge both the american/moroccan/french systems of promotion. This is counterproductive.

User 2 said this:
Research ethics

User 3 said this in response to user 2 :

This should not pose a problem if all researchers are trained in the same research ethics from the moment they become involved and if sanctions are applied to those who do not respect this ethical.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Trying to merge both the american/moroccan/french systems of promotion.
Classification of Answer: BARRIER
Argument: This is counterproductive.
Classification of Argument: PRO

Answer: Research ethics
Classification of Answer: BARRIER
Argument: This should not pose a problem if all researchers are trained in the same research ethics from the moment they become involved and if sanctions are applied to those who do not respect this ethical.
Classification of Argument: CON


#Test case 33

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''


User 1 said :

Do not make a difference between PhD student and Staff UM6P

User 2 said this in reponse to user 1 :
respect for levels PhD students and staff have distinct resource needs
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Do not make a difference between PhD student and Staff UM6P
Classification of Answer: BARRIER

Argument: respect for levels PhD students and staff have distinct resource needs
Classification of Argument: CON


#test case 34

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in the area of research?'''
theme = "research"
text = '''

User 1 said :

building a reputation which is a matter of work and time, taking by consideration the UM6P is a new born comparing to its peers in the fileds.

UM6P being a new university can limit the opportunity to publish in the best venues

User 2 said this in response to user 1 :

The reputation or the image if UM6P costs the university a lot money in purchasing equipment and consumables. On average the cost to sell to Um6p is 30% higher than the other universities across Morocco. This increase project costs.

User 3 said this to contribute to the conversation :

No, the reputation comes from doing quality work famous institutions often have a long history of research accomplishments, which may overshadow the limited track record of a new institutions.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: UM6P being a new university can limit the opportunity to publish in the best venues
Classification of Answer: BARRIER

Argument: The reputation or the image if UM6P costs the university a lot money in purchasing equipment and consumables. On average the cost to sell to Um6p is 30% higher than the other universities across Morocco. This increase project costs.
Classification of Argument: CON

Argument: No, the reputation comes from doing quality work famous institutions often have a long history of research accomplishments, which may overshadow the limited track record of a new institutions.
Classification of Argument: CON


#Test case 35

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we better succeed at recruiting research talent (PhD and profs)?'''
theme = "research"
text = '''
User 1 said :

By ensuring a better work environment the talents would come themselves to join the University and contribute to its development.

User 2 said this in response to user 1 :

A good work environemt can contribute to attrack good student but its not sUfficient

User 3 said to contribute to the conversation :
Safe space that provides freedom of research plus the financial support are the main attractions of great researches. having an environment of this nature will surely attract the best researches to this ecosystem
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: By ensuring a better work environment the talents would come themselves to join the University and contribute to its development.
Classification of Answer: SOLUTION
Argument: NONE

Answer: A good work environment can contribute to attract good students but it's not sufficient.
Classification of Answer: SOLUTION
Argument: NONE

Answer: Safe space that provides freedom of research plus the financial support are the main attractions of great researchers. Having an environment of this nature will surely attract the best researchers to this ecosystem.
Classification of Answer: SOLUTION
Argument: NONE


#Test case 36

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we better succeed at recruiting research talent (PhD and profs)?'''
theme = "research"
text = '''
User 1 said :

Use the unique combination atractive traits of research and highlight the fulfilment felt while working a research project.

User 2 said this in response to user 1 :

the fact that UM6p gives a decent living for a PHD student is attractive in itself, but it is necessary to highlight the most attractive trait which is the fulfilement felt from working in a research project.

User  3 said this in reponse to user 1 :
This can be done by putting in place programs that connect professors with potential students
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Use the unique combination attractive traits of research and highlight the fulfilment felt while working a research project.
Classification of Answer: SOLUTION

Argument: the fact that UM6p gives a decent living for a PHD student is attractive in itself, but it is necessary to highlight the most attractive trait which is the fulfilment felt from working in a research project.
Classification of Argument: PRO

Answer: putting in place programs that connect professors with potential students
Classification of Answer: SOLUTION

Argument: NONE


#Test case 37

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we overcome the barriers to success in research?'''
theme = "research"
text = '''
User 1 said :

Create a UM6P Analytics platform to allow all UM6P doctoral students to access any device and avoid customization of equipment
 the more tools at the disposal of student the more application possibilities will amerge.

'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Create a UM6P Analytics platform to allow all UM6P doctoral students to access any device and avoid customization of equipment
Classification of Answer: SOLUTION

Argument: the more tools at the disposal of student the more application possibilities will amerge.
Classification of Argument: PRO


#Test case 38

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we overcome the barriers to success in research?'''
theme = "research"
text = '''
User 1 said :

Provide a safe space for scientific staff and Phd students to work and suggest ideas.
The scientist needs to have the research facilities and a good environment to do his work properly
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Provide a safe space for scientific staff and PhD students to work and suggest ideas.
Classification of Answer: SOLUTION
Argument: The scientist needs to have the research facilities and a good environment to do his work properly.
Classification of Argument: PRO


#Test case 39

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6P’s success in the area of teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said :

the number of job offers that our graduates receive
Number of jobs received reflects the value of UM6P within the labor market

User 2 said  this in response to user 1:

The quantity does not reflect the quality of the offers
Are these offers consistent with the field and level of studies?
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: the number of job offers that our graduates receive
Classification of Answer: METRIC
Argument: Number of jobs received reflects the value of UM6P within the labor market
Classification of Argument: PRO

Answer: Are these offers consistent with the field and level of studies?
Classification of Answer: METRIC
Argument: The quantity does not reflect the quality of the offers
Classification of Argument: CON


#Test case 40

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said :

Achieve international accreditation for all our programs (undergrad and post grad)
 important KPI for recognition of UM6P programs

User 2 said this in response to user 1 :

Yes UM6P should follow the international universities courses and programs to have the international accreditation and also to have be able to do students exchange with different universities worldwide
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Achieve international accreditation for all our programs (undergrad and post grad)
Classification of Answer: METRIC
Argument: important KPI for recognition of UM6P programs
Classification of Argument: PRO

Answer: UM6P should follow the international universities courses and programs to have the international accreditation and also to have be able to do students exchange with different universities worldwide
Classification of Answer: SOLUTION
Argument: NONE


#Test case 41

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said :

Implementing an international curriculum allows students to gain a broader understanding of global issues, different cultural perspectives, and diverse approaches to learning. It helps foster a more inclusive and globally aware student body, enhancing the overall quality of education.

User 2 said this in response to user 1 :

Establishing partnerships with renowned international universities or institutions can demonstrate UM6Ps commitment to providing a global education. Collaborative research projects, joint degree programs, and faculty exchanges are potential measures of success in fostering international collaborations.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Implementing an international curriculum allows students to gain a broader understanding of global issues, different cultural perspectives, and diverse approaches to learning.
Classification of Answer: SOLUTION
Argument: It helps foster a more inclusive and globally aware student body, enhancing the overall quality of education.
Classification of Argument: PRO

Answer: Establishing partnerships with renowned international universities or institutions can demonstrate UM6Ps commitment to providing a global education.
Classification of Answer: SOLUTION
Argument: Collaborative research projects, joint degree programs, and faculty exchanges are potential measures of success in fostering international collaborations.
Classification of Argument: PRO


#Test case 42

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in teaching and student life?'''
theme = "teaching and student life?"
text = '''
User 1 said :


The lack of transparency and limited access to resources across different departments hinders students&apos ability to benefit from the expertise and opportunities available in various areas. By promoting transparency and facilitating cross-department collaboration, students can easily access the resources and support they need, enabling them to achieve their goals more effectively.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: The lack of transparency and limited access to resources across different departments hinders students' ability to benefit from the expertise and opportunities available in various areas.
Classification of Answer: BARRIER

Answer: By promoting transparency and facilitating cross-department collaboration, students can easily access the resources and support they need, enabling them to achieve their goals more effectively.
Classification of Answer: SOLUTION


#Test case 43

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said :

For UM6P to be successful in both teaching and student success, more students should be involved in decision-making. Only certain students are permitted to offer their opinions and contribute to decision-making. Not every student can contribute in the same way. The lack of connection with students, especially given the volume of emails, makes it difficult for them to identify relevant opportunities.&#160,,,,
,Absence for opportunities for students to have a social interactions and activities in the regi

on ,,,,availability of volunteering program; The absence of an opportunity is an opportunity in itself to be created,There is a program encouraing volunteering among UM6P students, to work with associations, and do some social work, the initiative is called SEP, we count today more than 40 students who participated UM6P offers many opportunities to students. There are departments who their activity is to help students launch clubs, social initiatives, and start-ups.... So maybe this is a lack of communication challenge rather than an absence of opportunities.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: More students should be involved in decision-making.
Classification of Answer: SOLUTION
Argument: Only certain students are permitted to offer their opinions and contribute to decision-making.
Classification of Argument: PRO

Answer: The lack of connection with students, especially given the volume of emails, makes it difficult for them to identify relevant opportunities.
Classification of Answer: BARRIER
Argument: NONE

Answer: Absence for opportunities for students to have social interactions and activities in the region.
Classification of Answer: BARRIER
Argument: NONE

Answer: The absence of an opportunity is an opportunity in itself to be created.
Classification of Answer: SOLUTION
Argument: There is a program encouraging volunteering among UM6P students, to work with associations, and do some social work, the initiative is called SEP, we count today more than 40 students who participated.
Classification of Argument: PRO

Answer: UM6P offers many opportunities to students.

#Test case 44

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in teaching and student life?'''
theme = "teaching and student life"
text = '''

User 1 said :

One idea to address the lack of communication and support in universities is to establish a student-led support team.
 This team would consist of dedicated students who are available to assist their peers in accessing various resources and providing guidance when needed.
  By creating such a team, students can benefit from improved communication channels and receive the necessary support to navigate their academic journey
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: One idea to address the lack of communication and support in universities is to establish a student-led support team.
Classification of Answer: SOLUTION
Argument: This team would consist of dedicated students who are available to assist their peers in accessing various resources and providing guidance when needed.
Classification of Argument: PRO
Argument: By creating such a team, students can benefit from improved communication channels and receive the necessary support to navigate their academic journey.
Classification of Argument: PRO


#Test case 45

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said :

Graduate Entrepreneurship Rate,Partnerships and Collaborations, Student Competitions and Awards, Alumni Engagement, Research Publications and Citations, Stakeholder SurveysIncubators and AcceleratorsSocial Impact Ventures and Start-up Success.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Graduate Entrepreneurship Rate
Classification of Answer: METRIC

Answer: Partnerships and Collaborations
Classification of Answer: METRIC

Answer: Student Competitions and Awards
Classification of Answer: METRIC

Answer: Alumni Engagement
Classification of Answer: METRIC

Answer: Research Publications and Citations
Classification of Answer: METRIC

Answer: Stakeholder Surveys
Classification of Answer: METRIC

Answer: Incubators and Accelerators
Classification of Answer: METRIC

Answer: Social Impact Ventures and Start-up Success
Classification of Answer: METRIC

Argument: NONE


#Test case 46

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said :

Develop the entrepreneurial spirit among students , create classroom spaces to exchange ideas or projects, facilitate financing for the different project leaders after having validated the business plan.

The proposal mainly reflects encouragement and not the measure of impact.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Develop the entrepreneurial spirit among students
Classification of Answer: SOLUTION

Answer: Create classroom spaces to exchange ideas or projects
Classification of Answer: SOLUTION

Answer: Facilitate financing for the different project leaders after having validated the business plan
Classification of Answer: SOLUTION

Argument: The proposal mainly reflects encouragement and not the measure of impact.
Classification of Argument: CON


#Test case 47

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''
User 1 said :

Number of startups created and funded in the UM6P ecosystem

User 2 said this in reponse to user 1 :

I would add to your idea one point: Real and Impactful startups.

User 3 made this contribution:
Entrepreneurial mindset development is more important. Startups are not sufficient for entrepreneurship to raise funds to set up an industrial or service business with an income statement and an investment plan and real commercial development to ensure the viability of entrepreneurs ale in time Following up the startup more important of the number of creation, because every day thousand of startup born but 1 or 2 continues
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Number of startups created and funded in the UM6P ecosystem
Classification of Answer: METRIC

Answer: Real and Impactful startups
Classification of Answer: METRIC

Answer: Entrepreneurial mindset development is more important.
Classification of Answer: SOLUTION

Argument: Startups are not sufficient for entrepreneurship to raise funds to set up an industrial or service business with an income statement and an investment plan and real commercial development to ensure the viability of entrepreneurs ale in time.
Classification of Argument: PRO

Argument: Following up the startup more important of the number of creation, because every day thousand of startup born but 1 or 2 continues.
Classification of Argument: PRO


#test case 48

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """


    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''

User 1 said :

Nb of startup launched in a period of time
That shows how dynamic the entrepreneurial ecosystem is.

Other factors such as the quality and viability of start-ups and their survival and growth rate are more relevant. Number but also thematic and business plan.

User 2 said this in response to user 1 :

The number of startups is an insufficient criterion; is it viable over time, economically, are we addressing an essential issue for Morocco, an example of sustainable development
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Nb of startup launched in a period of time
Classification of Answer: METRIC
Argument: That shows how dynamic the entrepreneurial ecosystem is.
Classification of Argument: PRO

Answer: Other factors such as the quality and viability of start-ups and their survival and growth rate are more relevant.
Classification of Answer: METRIC
Argument: Number but also thematic and business plan.
Classification of Argument: PRO

Answer: The number of startups is an insufficient criterion; is it viable over time, economically, are we addressing an essential issue for Morocco, an example of sustainable development
Classification of Answer: METRIC
Argument: NONE


#Test case 49

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''
User 1 said :

To measure UM6P success in entrepreneurship, we need to see if our startups are able to scale up to industrialization and being able to connect with other ecosystems (Moroccan &amp African ones)

This is the major indicator: torn over out of the OCP ecosystem

Scale up as well as startup which can long last over the longer periods of time
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: To measure UM6P success in entrepreneurship, we need to see if our startups are able to scale up to industrialization and being able to connect with other ecosystems (Moroccan & African ones)
Classification of Answer: METRIC

Argument: This is the major indicator: torn over out of the OCP ecosystem
Classification of Argument: PRO

Answer: Scale up as well as startup which can long last over the longer periods of time
Classification of Answer: METRIC


#Test case 50

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said :

Creation a Local entrepreneurship Ecosystem

Creating a local ecosystem is the best way to measure success in terms of entrepreneurship impact, I sense a massive interest in numbers like the total number of startups incubated and the amount of budget invested, I urge the community in this exercise to consider those as vanity metrics, the real, sustainable( in the time) value is creating an ecosystem and one of the best ways to measure our success in this is through measuring the interactions across the ecosystem we are creating

User 2 said  said this in response to user 1:

this idea is important so that we don&apost fall into the trap of thinking we are successful while our success is fragmented through some numbers, established ecosystems in silicon valley for example can rely on total numbers of investments and incubated startups , however in our case creating a solid ecosystem in Bengeurir is the most important and step ahead, even at the price of lower number in investemes

'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Creation a Local entrepreneurship Ecosystem
Classification of Answer: SOLUTION
Argument: Creating a local ecosystem is the best way to measure success in terms of entrepreneurship impact
Classification of Argument: PRO

Answer: Creating a local ecosystem is the best way to measure success in terms of entrepreneurship impact
Classification of Answer: METRIC
Argument: I sense a massive interest in numbers like the total number of startups incubated and the amount of budget invested, I urge the community in this exercise to consider those as vanity metrics, the real, sustainable( in the time) value is creating an ecosystem and one of the best ways to measure our success in this is through measuring the interactions across the ecosystem we are creating
Classification of Argument: PRO

Answer: Creating a solid ecosystem in Bengeurir is the most important and step ahead, even at the price of lower number in investments
Classification of Answer: SOLUTION
Argument: this idea is importa

#Test case 51

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said :

Number of IPs based on which startups are created, Savings to industriels using solutions offered by researchers and engineers from UM6P

Value creation: jobs, increase competitiveness

User 2 said this in response to user 1 :

the number of IPs my be a vanity metric that dosent represnt the vlaue of the impact

IPs are important mybe hard to translate to value
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Number of IPs based on which startups are created
Classification of Answer: METRIC

Argument: the number of IPs my be a vanity metric that dosent represnt the vlaue of the impact
Classification of Argument: CON

Argument: IPs are important mybe hard to translate to value
Classification of Argument: CON

Answer: Savings to industriels using solutions offered by researchers and engineers from UM6P
Classification of Answer: METRIC

Answer: Value creation: jobs, increase competitiveness
Classification of Answer: METRIC


#Test case 52

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''

User 1 :
 Number of listed companies generated via UM6P that are quoted on a stock exchange. If UM6P create companies and contribute direcrly to GDP in Morocco and Africa hence it will have a social impact and improving livelihood towards zero poverty  and other SDGs.


User 2 said this in reponse to user 1:

This number is far fetch and we will have to wait for couple of years to be able to start monitoring this number and mesure our success according to it, what we need right now is another number to measure our progress twords this number .
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Number of listed companies generated via UM6P that are quoted on a stock exchange.
Classification of Answer: METRIC
Argument: If UM6P create companies and contribute directly to GDP in Morocco and Africa hence it will have a social impact and improving livelihood towards zero poverty and other SDGs.
Classification of Argument: PRO

Answer: what we need right now is another number to measure our progress towards this number.
Classification of Answer: METRIC
Argument: This number is far fetch and we will have to wait for couple of years to be able to start monitoring this number and measure our success according to it.
Classification of Argument: CON


#Test case 53

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''
User 1 :

To be successful, our impact should be seen first on the region where we are (Benguerir). We cannot target Morocco &amp Africa without having a visible impact on rehamna region.  If we fail to impact the local community in Rhamna then we will definately fail to impact any larger scale, creating the local ecosystem in bengeurir is key and progress will be based on it .

User 2 in response to user 1:

Our impact should go beyond Rhamna. Indeed Rhamna is a living lab, but developped solutions should target other regions especialy that the UM6P is now on different cities and continents.  A business by nature obeys a market.

User 3 contributing to their conversation:

Of course the Rhamna region must benefit from development if possible but a business first corresponds to a market by nature for marketing to define the most appropriate market whether local or not.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: our impact should be seen first on the region where we are (Benguerir). We cannot target Morocco &amp Africa without having a visible impact on rehamna region. If we fail to impact the local community in Rhamna then we will definately fail to impact any larger scale, creating the local ecosystem in bengeurir is key and progress will be based on it.
Classification of Answer: METRIC
Argument: NONE
Classification of Argument: NONE

Answer: Our impact should go beyond Rhamna. Indeed Rhamna is a living lab, but developped solutions should target other regions especialy that the UM6P is now on different cities and continents.
Classification of Answer: SOLUTION
Argument: NONE
Classification of Argument: NONE

Answer: A business by nature obeys a market.
Classification of Answer: SOLUTION
Argument: NONE
Classification of Argument: NONE

Answer: a business first corresponds to a market by nature for marketing to define the most appropriate market whether local or not.
Classification of 

#Test case 54

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1:

Number of smallholder farmers impacted : Biggest college at UM6P is SAES, hence the direct impact on smallholders in Morocco and Africa should be a crucial KPI to measure the university impact.

User 2 in response to user 1:

We should set the number of farmers as a KPI and also how did we change the farmers livelihood? The number is good, the impact at the level of each farmer is even more important Small producers what a significant impact.

User 3 contributing to their conversation:

50% of the African continent works in the agricultural sector from production to sale on the markets...sometimes small technological improvements human social educational and of course climatic and average soils of substances and plant production and humans have an exponential impact on the future of the continent

'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Number of smallholder farmers impacted
Classification of Answer: METRIC
Argument: Biggest college at UM6P is SAES, hence the direct impact on smallholders in Morocco and Africa should be a crucial KPI to measure the university impact.
Classification of Argument: PRO

Answer: We should set the number of farmers as a KPI and also how did we change the farmers livelihood?
Classification of Answer: METRIC
Argument: The number is good, the impact at the level of each farmer is even more important.
Classification of Argument: PRO

Answer: Small producers what a significant impact.
Classification of Answer: METRIC
Argument: NONE

Answer: 50% of the African continent works in the agricultural sector from production to sale on the markets...sometimes small technological improvements human social educational and of course climatic and average soils of substances and plant production and humans have an exponential impact on the future of the continent
Classification of Answer: SOLUTION
Ar

#Test case 55

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the best ways to measure UM6Ps success in the area of entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''
User 1:

The impact or possible future impact of my research, projects, and actions, what actions i am taking or will take in the future that could have an impact

User 2 in response to user 1:

Most research done in Morocco has no real impact in society which is a waste of money but Um6p must be different  Action based research and project including give a PhD degree based on relevant startup and give points on KPI to teachers for enabling startups  scientific basis and innovation centric .
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: give a PhD degree based on relevant startup
Classification of Answer: SOLUTION
Argument: NONE
Classification of Argument: NONE

Answer: give points on KPI to teachers for enabling startups scientific basis and innovation centric
Classification of Answer: SOLUTION
Argument: NONE
Classification of Argument: NONE


#Test case 56

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship and social Impact?'''
theme = "entrepreneurship and social Impact?"
text = '''

User 1 :

Diversity : Diversity to have a holistic approach to the problems to be solved

User 2 in response to user 1:

Diversity is the problem, because we should know the problem and find a solution  not just speak about diversity and loose time and ideas
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Diversity
Classification of Answer: SOLUTION
Argument: Diversity to have a holistic approach to the problems to be solved
Classification of Argument: PRO

Answer: Diversity is the problem
Classification of Answer: BARRIER
Argument: because we should know the problem and find a solution not just speak about diversity and loose time and ideas
Classification of Argument: CON


#Test case 57

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 :


Lack of skilled monitoring committee to follow-up ideas

User 2 in response to user 1:

Monitoring is absolute necessary in every stages of idea and startup  Mentorant experiences

User 3 contributed to the conversation:
Senior business experience is important to guide young people, real business leaders who have managed or set up a business.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Lack of skilled monitoring committee to follow-up ideas
Classification of Answer: BARRIER

Answer: Monitoring is absolute necessary in every stages of idea and startup Mentorant experiences
Classification of Answer: SOLUTION

Argument: Senior business experience is important to guide young people, real business leaders who have managed or set up a business.
Classification of Argument: PRO


#Test case 58

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship and social Impact?'''
theme = "entrepreneurship and social Impact?"
text = '''

User 1 :

projects and collaborations made focus only on research, no impact or entrepreneurship deliverables are considered

User 2 in response to user 1:
At um6p we believe too much that the researcher is the best manager the best salesman the best entrepreneur state of mind to change to have the diversity of profiles and skills.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: projects and collaborations made focus only on research, no impact or entrepreneurship deliverables are considered
Classification of Answer: BARRIER

Answer: At um6p we believe too much that the researcher is the best manager the best salesman the best entrepreneur state of mind to change to have the diversity of profiles and skills.
Classification of Answer: BARRIER


#Test case 59

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact?"
text = '''
User 1 :

Not fully leveraging the potential of its people.
Leveraging People management
To improve the potential of an entrepreneur, in particular a startup researcher, management is required and even management by experienced business leaders
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Not fully leveraging the potential of its people.
Classification of Answer: BARRIER

Answer: Leveraging People management
Classification of Answer: SOLUTION

Argument: To improve the potential of an entrepreneur, in particular a startup researcher, management is required and even management by experienced business leaders.
Classification of Argument: PRO


#Test case 60

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said :

Transend Political barriers and not having direct impact on society

At the opposit of other universities locally, UM6P has a unique business model where it does  have the capacity to connect research to society directly.
Hence, it can convey its research and knoweledge to direct action.
Just to illustrate this idea, Sustainable Agriculture College, instead of only doing research it may have its own scalable production farm that can innovate in all aspect of national and african agriculture.
 In such model, it is research by proving as the &quotlearning by doing&quot and UM6P can then streamline research ideas into real life, build its credibility and redifine excellence by pushing the boundaries of possible. E.g. Les Domaines UM6P  a leader in agricol production in Morocco and having farms in many African countries, employing thousands, generating millions and providing food security to thousands...(that was just an example)
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Transend Political barriers and not having direct impact on society
Classification of Answer: BARRIER

Answer: UM6P has a unique business model where it does have the capacity to connect research to society directly.
Classification of Answer: SOLUTION

Argument: Hence, it can convey its research and knoweledge to direct action.
Classification of Argument: PRO

Answer: Sustainable Agriculture College, instead of only doing research it may have its own scalable production farm that can innovate in all aspect of national and african agriculture.
Classification of Answer: SOLUTION

Argument: In such model, it is research by proving as the "learning by doing" and UM6P can then streamline research ideas into real life, build its credibility and redifine excellence by pushing the boundaries of possible.
Classification of Argument: PRO

Argument: Les Domaines UM6P a leader in agricol production in Morocco and having farms in many African countries, employing thousands, generating milli

#Test case 61

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''

User 1 :

Researchers who think that entrepreneurship is a waste of time and can ruin their career

User 2 said this in response to user 1:
Research with no added value to people will remain only a paper, there should be real impact financially and socially on livelihood Action based research translation into Startup  researcher are not made for entrepreneurship and they should be open to collaborate with business people.

User 3 added to the conversation:
Having an entrepreneurship mindset keeps us always relevent Careers are better enhanced by having an enterpreneuship mindset.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Researchers who think that entrepreneurship is a waste of time and can ruin their career
Classification of Answer: BARRIER

Answer: Research with no added value to people will remain only a paper, there should be real impact financially and socially on livelihood
Classification of Answer: BARRIER

Argument: Action based research translation into Startup researcher are not made for entrepreneurship and they should be open to collaborate with business people.
Classification of Argument: PRO

Answer: Having an entrepreneurship mindset keeps us always relevant
Classification of Answer: SOLUTION

Argument: Careers are better enhanced by having an entrepreneurship mindset.
Classification of Argument: PRO


#Test case 62

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 said:

Not daring enough!

In a dynamic and fast changing world, UM6P should have an agile, yet, efficient approach to dare exploring new frontiers and ideas. However, it should have to ssess quickly and set up correction measures when necessary. &#10&lthr&gtNon daring mindsets, fear of adventures and risks, comfort zone laziness&lthr&gt,Right people in the area of Entrepreneurship is the need of the hour
Researchers are more career driven than impact oriented
Failing to embrace and translate collective intelligence from a work paradigm to a day to day practice,,More interdepartmental collaboration and sharing of research and practices zmwith everyone
Level of communication, make participation and integration of entrepreneurship more fluid and simpler. especially the files &#224; submit ,,,,The mentality, this is good enough, no attention to details .


User 2 said this in response to user 1:
The product we deliver is not finished but you can customize it as you please

User 3 also said this:
The product shoud be sold finished, ready to use, therefore all the cost should be included and quantified.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Not daring enough!
Classification of Answer: BARRIER

Answer: Non daring mindsets, fear of adventures and risks, comfort zone laziness
Classification of Answer: BARRIER

Answer: Right people in the area of Entrepreneurship is the need of the hour
Classification of Answer: SOLUTION

Answer: Researchers are more career driven than impact oriented
Classification of Answer: BARRIER

Answer: Failing to embrace and translate collective intelligence from a work paradigm to a day to day practice
Classification of Answer: BARRIER

Answer: More interdepartmental collaboration and sharing of research and practices with everyone
Classification of Answer: SOLUTION

Answer: Level of communication, make participation and integration of entrepreneurship more fluid and simpler, especially the files to submit
Classification of Answer: SOLUTION

Answer: The mentality, this is good enough, no attention to details
Classification of Answer: BARRIER

Argument: However, it should have to assess quickl

#Test case 63

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''What are the biggest challenges to UM6P achieving success in entrepreneurship & social Impact?'''
theme = "entrepreneurship & social Impact"
text = '''
User 1 :

selfishness : we need to develop the collective spirit of work


Academic Ego  is a critical factor in hindrance of Entrepreneurship.
reseachers should stop looking only at their CV because an entire community is waiting for their outputs based on sciences and experiments.
A lot specially by people in research fields we have lot of expert but everyone keep what he have for him self this what others called selfishness.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: selfishness : we need to develop the collective spirit of work
Classification of Answer: BARRIER

Answer: Academic Ego is a critical factor in hindrance of Entrepreneurship.
Classification of Answer: BARRIER

Argument: reseachers should stop looking only at their CV because an entire community is waiting for their outputs based on sciences and experiments.
Classification of Argument: PRO

Argument: A lot specially by people in research fields we have lot of expert but everyone keep what he have for him self this what others called selfishness.
Classification of Argument: PRO


#Test case 64

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in research?'''
theme = "research"
text = '''

User 1 said:
 To overcome barriers to success in research, we should find applications for theoretical research.
Applications of theoretical research increase its societal impact This is important for impact.

User 2 said this in response to user 1:

Prioritizing applications may limit pure, exploratory theoretical research Starting from the need to establish research projects in order to ensure there are applications of the research findings.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: To overcome barriers to success in research, we should find applications for theoretical research.
Classification of Answer: SOLUTION
Argument: Applications of theoretical research increase its societal impact. This is important for impact.
Classification of Argument: PRO

Answer: Prioritizing applications may limit pure, exploratory theoretical research.
Classification of Answer: BARRIER
Argument: Starting from the need to establish research projects in order to ensure there are applications of the research findings.
Classification of Argument: CON


#Test case 65

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = ''' How can we achieve success in research?'''
theme = "research"
text = '''
User 1 said:
In my opinion, success in research depends above all on working with tangible models.
The practical application of the theories found in research, and the thinking and execution of our researchers, are the best methods for ensuring success in different sectors.
In the final phase, success can be measured by the adoption of our research topics by others.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: working with tangible models
Classification of Answer: SOLUTION

Argument: The practical application of the theories found in research, and the thinking and execution of our researchers, are the best methods for ensuring success in different sectors.
Classification of Argument: PRO

Answer: the adoption of our research topics by others
Classification of Answer: METRIC

Argument: NONE


#Test case 66

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = ''' How can we achieve success in research?'''
theme = "research"
text = '''
User 1 said:


In my opinion, to push research further, we should diversify subjects and invest as much as possible in the world of research. The more we have the chance to dig, the more information we get.
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: to push research further, we should diversify subjects and invest as much as possible in the world of research.
Classification of Answer: SOLUTION
Argument: The more we have the chance to dig, the more information we get.
Classification of Argument: PRO


#Test case 68

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """


    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 :
Providing the students with clear understanding of how the corporate and/or the research world are, and how they could provide an added value with their abilities and knowledge.

This prepares the students for the real world
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Providing the students with clear understanding of how the corporate and/or the research world are, and how they could provide an added value with their abilities and knowledge.
Classification of Answer: SOLUTION

Argument: This prepares the students for the real world.
Classification of Argument: PRO


#Test case 69

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in teaching and student life?'''
theme = "teaching and student life"
text = '''

User 1 said:

Gamify the teaching experience when the subject nature allows it to engage students and make learning fun through challenges with rewards and achievements

User 2 said this in response to user 1 :

Gamification enhances engagement and learning effectiveness

User 3 said this to contribute to the conversation :

42 network is proof that the gamification of the learning experience is a great model
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Gamify the teaching experience when the subject nature allows it to engage students and make learning fun through challenges with rewards and achievements
Classification of Answer: SOLUTION
Argument: Gamification enhances engagement and learning effectiveness
Classification of Argument: PRO
Argument: 42 network is proof that the gamification of the learning experience is a great model
Classification of Argument: PRO


#Test case 70

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said:
Implement cohesive teaching guidelines

Students emulate structured, effective teacher behavior .
'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: Implement cohesive teaching guidelines
Classification of Answer: SOLUTION

Argument: Students emulate structured, effective teacher behavior.
Classification of Argument: PRO


#Test case 71

In [ ]:
def extract_points_find_clusters_and_analyze_arguments(theme, text):
    # Step 1: Extract points
    prompt_analysis = f"""
    The following  4 examples are from conversations scripts for different questions with their annotations and how we arrived at the annoations:

Example 1 :

Conversation transcripts :
User 1 said :
By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.
Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO
·  Reasoning: This argument supports the main  first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.
Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO
Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.
Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON
·  Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.

Example 2:

Question : How can we achieve success in research?
Conversation transcripts :

User 1 said :

By sponsoring students to attend research program. It will help students to learn, present their research, receive feedback and publish in top journals.
It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student can have grown to win grants for sponsorship from outside. On the other hand, it can be abused if not proper tracked.

Here is the annotated answer:

Answer: Sponsor students to attend research programs
Classification of Answer: Solutions
Reasoning: This statement directly suggests a method to achieve success in research, making it a solution-oriented response.

Argument 1: It will help students learn how to present research, receive feedback, and publish in top journals.
Classification of Argument 1 : PRO

Reasoning: This argument supports the first answer by highlighting the benefits of sponsoring students, which aligns with achieving success in research.

Argument 2: It is financially sustainable because even though UM6P may spend more when the student is just coming up, later on the student would have grown to win grants for sponsorship from outside.
Classification Argument 2: PRO

Reasoning: This argument provides a financial justification, arguing that initial investment will be compensated by future external grants, thus supporting the feasibility of the solution.

Argument 3: It can be abused if not properly tracked.
Classification of Argument 3: CON

Reasoning: This argument points out a potential downside or risk associated with the solution, which is important to consider for a balanced view.





Example 3:

Question : How can we achieve success in research?

Conversation transcript :

User 1 said :

A better approach to methodology class. One of the problems is that way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter. It is key for researchers since the first years of their training to know the importance of it. (This statement is particularly true in the case of social sciences)

Annotated answers

Answer : the way methodology is taught. The most boring way possible with no creativity way and lack of explaining the importance of the matter.
Classification of Answer: barriers

Reasoning: This statement highlights the current issues with the methodology classes, identifying them as obstacles that hinder success in research.

Answer : better approach to methodology class
Classification of Answer: solutions

Reasoning: This statement suggests an improvement to overcome the identified barriers, proposing a solution-oriented approach to teaching methodology.

Argument :  It is key for researchers since the first years of their training to know the importance of it, particularly in the case of social sciences
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by emphasizing the significance of understanding methodology from the early stages of research training, particularly in social sciences, thus reinforcing the need for a better approach.




EXAMPLE 4 :
Question : How can we achieve success in research?

Conversation transcript :

We can achieve success in research by focusing on researches that solve real world problem. A research that solves a real world problem will in turn bring more funding.

Annotated answer:
Answer : We can achieve success in research by focusing on researches that solve real world problem.
Classification of Answer: SOLUTION

Reasoning: This statement provides a direct method for achieving success in research by emphasizing the importance of addressing real-world issues, thus presenting a solution-oriented approach

Argument: A research that solves a real world problem will in turn bring more funding.
Classification of Argument: PRO

Reasoning: This argument supports the suggested solution by highlighting a positive outcome (increased funding) that results from focusing on real-world problems, reinforcing the benefits of the proposed approach.
    This {text} contains answer or answers to the {Question}, sometimes with arguments for (PRO)  or against the answers (CON) this answer(s).
    Find all the answer(s) directly answering the question.
    Then find all arguments against or for the answer(s).
     NB: CON arguments are sentences that give reason against the answers WHILE PRO arguments give reasons to solidify the answers.
    NB : That a sentence is negative does not automatically make it a CON argument, check it carefuly before you decide..
    Then classify the argument into PRO or CON.
    Make sure to only return output strictly from the text.
    Then classify each answers into:
    - SOLUTION: a way to achieve success in {theme}.
    - METRIC: a way to measure the success of {theme}.
    - BARRIER: a barrier to the success of {theme}.
    For each point, give me the type of point (SOLUTION, METRIC, BARRIER) followed by, IN THE SAME LINE, all the text describing that point.
    Make sure to only return answers strictly from the text.
    If there are no points/arguments in the text, answer NONE.

 The output should be in the following format:

      Answer:
      Classification of Answer:
      Argument :
      Classification of Argument :

     Repeat the same format if there are more than one answers in the text.

    NB: There might be none, single or multiple answers/arguments  in the text, same for the arguments.

    """

    response_analysis = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": "You are a helpful assistant that is very meticulous and reads every text line by line and finds answer(s) to the questions wiht their argument in the given text"},
          {"role": "user", "content": prompt_analysis},
      ],
      max_tokens=1024,
      temperature=0,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    feedback = response_analysis.choices[0].message.content.split('\n')

    # Print each feedback item on a new line
    for item in feedback:
        print(item)  # This ensures each item is on a new line

# Example usage
Question = '''How can we achieve success in teaching and student life?'''
theme = "teaching and student life"
text = '''
User 1 said:

To improve teaching and student life at UM6P, we can promote international student exchange programs. This will cultivate a diverse and globally conscious student community, thereby elevating the standard of education

'''
extract_points_find_clusters_and_analyze_arguments(theme, text)

Answer: To improve teaching and student life at UM6P, we can promote international student exchange programs.
Classification of Answer: SOLUTION
Argument: This will cultivate a diverse and globally conscious student community, thereby elevating the standard of education.
Classification of Argument: PRO
